In [7]:
import math
def comparison(testlabel, resultslabel):

    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for row1 in range(len(resultslabel)):
        if resultslabel[row1] < 0.5:
            resultslabel[row1] = 0
        else:
            resultslabel[row1] = 1

    for row2 in range(len(testlabel)):

        if testlabel[row2] == 1 and testlabel[row2] == resultslabel[row2]:
            TP = TP + 1
        if testlabel[row2] == 0 and testlabel[row2] != resultslabel[row2]:
            FP = FP + 1
        if testlabel[row2] == 0 and testlabel[row2] == resultslabel[row2]:
            TN = TN + 1
        if testlabel[row2] == 1 and testlabel[row2] != resultslabel[row2]:
            FN = FN + 1

    # ACC：accuracy
    if TP + TN + FP + FN != 0:
        ACC = (TP + TN) / (TP + TN + FP + FN)
    else:
        ACC = 999999

    # F1 score：is the harmonic mean of precision and sensitivity
    if TP + FP + FN != 0:
        F1 = (2 * TP) / (2 * TP + FP + FN)
    else:
        F1 = 999999

    # MCC：Matthews correlation coefficient
    if (TP + FP) * (TP + FN) * (TN + FP) * (TN + FN) != 0:
        MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    else:
        MCC = 999999

    return TP, FP, TN, FN, ACC, F1, MCC

In [6]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPool1D, Flatten, Dropout, Conv1D,Reshape,Input,AveragePooling1D,LSTM,Attention,MultiHeadAttention,MaxPooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras import regularizers,Model,optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.losses import binary_crossentropy, hinge
callback4 = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,
                                             verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
                 callbacks.ModelCheckpoint(filepath=r'./model/bestmodel', monitor='val_accuracy', verbose=0,mode = 'max',
                                           save_best_only=True, save_weights_only=False)]
tf.random.set_seed(123)
np.random.seed(123)

lu = 100
f = open('./ecc/arabidopsis/'+str(lu)+'/arabidopsis.csv')
train_data = []
label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    dd = []
    for j in tt[0].upper():
        if(j == 'A'):
            dd.append([0,0,0,1])
        elif (j == 'G'):
            dd.append([0,0,1,0])
        elif (j == 'C'):
            dd.append([0,1,0,0])
        elif (j == 'T'):
            dd.append([1,0,0,0])
        else:
            dd.append([0,0,0,0])
    while(len(dd) < lu*2):
        dd.append([0,0,0,0])
    train_data.append(dd)

    label.append(int(tt[1]))



train_data = np.array(train_data)
label = np.array(label)





train_data,test_data,label,test_label=train_test_split(train_data,label,test_size=0.1,random_state=3753,stratify=label)

test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
label = np.array(label)
print(train_data.shape)
print(test_data.shape)




inputs1 = Input(shape=(lu*2,4),name='1st_input')

print(fn.shape)






x_mod = Conv1D(128,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)

x_mod = Conv1D(64,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x_mod)

x_mod = MaxPooling1D(2)(x_mod)
x_mod = Dropout(0.1)(x_mod)
# x_mod = BatchNormalization()(x_mod)





co1 = Conv1D(x_mod.shape[1],4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
co2 = Conv1D(x_mod.shape[1],16, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co3 = Conv1D(247,6, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co4 = Conv1D(247,8, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co5 = Conv1D(247,10, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
pool1 = GlobalMaxPooling1D()(co1)
pool2 = GlobalMaxPooling1D()(co2)
# pool3 = GlobalMaxPooling1D()(co3)
# pool4 = GlobalMaxPooling1D()(co4)
# pool5 = GlobalMaxPooling1D()(co5)

# 使用FPN结构合并多尺度特征
fn = tf.add(pool1, pool2)
fn = Reshape((x_mod.shape[1], -1))(fn)


x_mod = Concatenate()([fn, x_mod])
x_mod = LSTM(64,return_sequences=True)(x_mod)


x_mod = Dropout(0.1)(x_mod)


x_mod = Flatten()(x_mod)
x_mod = Dropout(0.1)(x_mod)
x_mod = Dense(64, activation='sigmoid')(x_mod)
out = Dense(1, activation='sigmoid')(x_mod)

model = Model(inputs=[inputs1], outputs=[out])#合成模型


# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 交叉熵损失
    cross_entropy_loss = binary_crossentropy(y_true, y_pred)
    
    # Hinge Cross-Entropy Loss
    hinge_loss = tf.losses.hinge(y_true, y_pred)
    
    # 将两个损失相加
    total_loss = 0.95*cross_entropy_loss + 0.05*hinge_loss
    
    return total_loss




model.compile(loss=custom_loss,
              optimizer=optimizers.Adam(learning_rate=0.001,beta_1=0.9, beta_2=0.888, epsilon=1e-08),metrics=['accuracy'],
              )



train_history = model.fit(train_data,
                          label,
                          validation_split=0.1,
                          epochs=20,
                          batch_size=64,
                          verbose=1,
                          shuffle=True,
                          callbacks = callback4)


model.load_weights('./model/bestmodel')
pl = model.predict(test_data)
p_test_label = []

count = 0
for i in range(len(pl)):
    if(pl[i] >= 0.5):
        p_test_label.append(1)
    else:
        p_test_label.append(0)
# print(p_test_label)
TP, FP, TN, FN, ACC, F1, MCC = comparison(p_test_label, test_label)
print('PLeccNET')
print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN)
print('ACC:', ACC, 'F1:', F1, 'MCC:', MCC) 



(35980, 200, 4)
(3998, 200, 4)
(None, 247, 1)
Epoch 1/20
506/506 [==============================] - ETA: 0s - loss: 0.5389 - accuracy: 0.7541INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 19s 27ms/step - loss: 0.5389 - accuracy: 0.7541 - val_loss: 0.5091 - val_accuracy: 0.7685 - lr: 0.0010
Epoch 2/20
504/506 [============================>.] - ETA: 0s - loss: 0.4497 - accuracy: 0.8195INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 13s 25ms/step - loss: 0.4493 - accuracy: 0.8197 - val_loss: 0.4280 - val_accuracy: 0.8307 - lr: 0.0010
Epoch 3/20
505/506 [============================>.] - ETA: 0s - loss: 0.4234 - accuracy: 0.8326INFO:tensorflow:Assets written to: ./model/bestmodel/assets


...

506/506 [==============================] - 13s 25ms/step - loss: 0.3496 - accuracy: 0.8735 - val_loss: 0.4057 - val_accuracy: 0.8438 - lr: 0.0010
Epoch 13/20
504/506 [============================>.] - ETA: 0s - loss: 0.3451 - accuracy: 0.8760INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 13s 25ms/step - loss: 0.3451 - accuracy: 0.8760 - val_loss: 0.3917 - val_accuracy: 0.8524 - lr: 0.0010
Epoch 14/20
506/506 [==============================] - 8s 16ms/step - loss: 0.3408 - accuracy: 0.8770 - val_loss: 0.4283 - val_accuracy: 0.8335 - lr: 0.0010
Epoch 15/20
506/506 [==============================] - 8s 15ms/step - loss: 0.3370 - accuracy: 0.8796 - val_loss: 0.4265 - val_accuracy: 0.8319 - lr: 0.0010
Epoch 16/20
506/506 [==============================] - 8s 16ms/step - loss: 0.3352 - accuracy: 0.8810 - val_loss: 0.4158 - val_accuracy: 0.8416 - lr: 0.0010
Epoch 17/20
506/506 [==============================] - 8s 16ms/step - loss: 0.3263 - accuracy: 0.8828 - val_loss: 0.4052 - val_accuracy: 0.8374 - lr: 0.0010
Epoch 18/20
506/506 [==============================] - 8s 16ms/step - loss: 0.3209 - accuracy: 0.8880 - val_loss: 0.4473 - val_accuracy: 0.8307 - lr: 0.0010
Epoch 19/20
506/506 [==============================] - 8s 16ms/step -

2023-09-19 10:07:58.323492: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./model/bestmodel: FAILED_PRECONDITION: model/bestmodel; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


125/125 [==============================] - 1s 5ms/step
PLeccNET
TP: 1710 FP: 289 TN: 1676 FN: 323
ACC: 0.8469234617308654 F1: 0.8482142857142857 MCC: 0.6939473064586114


In [8]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPool1D, Flatten, Dropout, Conv1D,Reshape,Input,AveragePooling1D,LSTM,Attention,MultiHeadAttention,MaxPooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras import regularizers,Model,optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.losses import binary_crossentropy, hinge
callback4 = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,
                                             verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
                 callbacks.ModelCheckpoint(filepath=r'./model/bestmodel', monitor='val_accuracy', verbose=0,mode = 'max',
                                           save_best_only=True, save_weights_only=False)]
tf.random.set_seed(123)
np.random.seed(123)

lu = 150
f = open('./ecc/arabidopsis/'+str(lu)+'/arabidopsis.csv')
train_data = []
label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    dd = []
    for j in tt[0].upper():
        if(j == 'A'):
            dd.append([0,0,0,1])
        elif (j == 'G'):
            dd.append([0,0,1,0])
        elif (j == 'C'):
            dd.append([0,1,0,0])
        elif (j == 'T'):
            dd.append([1,0,0,0])
        else:
            dd.append([0,0,0,0])
    while(len(dd) < lu*2):
        dd.append([0,0,0,0])
    train_data.append(dd)

    label.append(int(tt[1]))



train_data = np.array(train_data)
label = np.array(label)





train_data,test_data,label,test_label=train_test_split(train_data,label,test_size=0.1,random_state=3753,stratify=label)

test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
label = np.array(label)
print(train_data.shape)
print(test_data.shape)




inputs1 = Input(shape=(lu*2,4),name='1st_input')








x_mod = Conv1D(128,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)

x_mod = Conv1D(64,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x_mod)

x_mod = MaxPooling1D(2)(x_mod)
x_mod = Dropout(0.1)(x_mod)
# x_mod = BatchNormalization()(x_mod)





co1 = Conv1D(x_mod.shape[1],4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
co2 = Conv1D(x_mod.shape[1],16, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co3 = Conv1D(247,6, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co4 = Conv1D(247,8, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co5 = Conv1D(247,10, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
pool1 = GlobalMaxPooling1D()(co1)
pool2 = GlobalMaxPooling1D()(co2)
# pool3 = GlobalMaxPooling1D()(co3)
# pool4 = GlobalMaxPooling1D()(co4)
# pool5 = GlobalMaxPooling1D()(co5)

# 使用FPN结构合并多尺度特征
fn = tf.add(pool1, pool2)
fn = Reshape((x_mod.shape[1], -1))(fn)


x_mod = Concatenate()([fn, x_mod])
x_mod = LSTM(64,return_sequences=True)(x_mod)


x_mod = Dropout(0.1)(x_mod)


x_mod = Flatten()(x_mod)
x_mod = Dropout(0.1)(x_mod)
x_mod = Dense(64, activation='sigmoid')(x_mod)
out = Dense(1, activation='sigmoid')(x_mod)

model = Model(inputs=[inputs1], outputs=[out])#合成模型


# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 交叉熵损失
    cross_entropy_loss = binary_crossentropy(y_true, y_pred)
    
    # Hinge Cross-Entropy Loss
    hinge_loss = tf.losses.hinge(y_true, y_pred)
    
    # 将两个损失相加
    total_loss = 0.95*cross_entropy_loss + 0.05*hinge_loss
    
    return total_loss




model.compile(loss=custom_loss,
              optimizer=optimizers.Adam(learning_rate=0.001,beta_1=0.9, beta_2=0.888, epsilon=1e-08),metrics=['accuracy'],
              )



train_history = model.fit(train_data,
                          label,
                          validation_split=0.1,
                          epochs=20,
                          batch_size=64,
                          verbose=1,
                          shuffle=True,
                          callbacks = callback4)


model.load_weights('./model/bestmodel')
pl = model.predict(test_data)
p_test_label = []

count = 0
for i in range(len(pl)):
    if(pl[i] >= 0.5):
        p_test_label.append(1)
    else:
        p_test_label.append(0)
# print(p_test_label)
TP, FP, TN, FN, ACC, F1, MCC = comparison(p_test_label, test_label)
print('PLeccNET')
print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN)
print('ACC:', ACC, 'F1:', F1, 'MCC:', MCC) 



(35980, 300, 4)
(3998, 300, 4)
Epoch 1/20
506/506 [==============================] - ETA: 0s - loss: 0.5211 - accuracy: 0.7718INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 19s 29ms/step - loss: 0.5211 - accuracy: 0.7718 - val_loss: 0.4538 - val_accuracy: 0.8107 - lr: 0.0010
Epoch 2/20
504/506 [============================>.] - ETA: 0s - loss: 0.3964 - accuracy: 0.8532INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 14s 27ms/step - loss: 0.3960 - accuracy: 0.8533 - val_loss: 0.3758 - val_accuracy: 0.8644 - lr: 0.0010
Epoch 3/20
503/506 [============================>.] - ETA: 0s - loss: 0.3699 - accuracy: 0.8623INFO:tensorflow:Assets written to: ./model/bestmodel/assets


...

506/506 [==============================] - 14s 27ms/step - loss: 0.2861 - accuracy: 0.9065 - val_loss: 0.3447 - val_accuracy: 0.8819 - lr: 0.0010
Epoch 13/20
503/506 [============================>.] - ETA: 0s - loss: 0.2888 - accuracy: 0.9044INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 14s 27ms/step - loss: 0.2888 - accuracy: 0.9045 - val_loss: 0.3431 - val_accuracy: 0.8833 - lr: 0.0010
Epoch 14/20
506/506 [==============================] - 9s 18ms/step - loss: 0.2796 - accuracy: 0.9097 - val_loss: 0.4470 - val_accuracy: 0.8346 - lr: 0.0010
Epoch 15/20
506/506 [==============================] - 9s 18ms/step - loss: 0.2716 - accuracy: 0.9128 - val_loss: 0.4122 - val_accuracy: 0.8605 - lr: 0.0010
Epoch 16/20
506/506 [==============================] - 9s 18ms/step - loss: 0.2708 - accuracy: 0.9134 - val_loss: 0.3820 - val_accuracy: 0.8602 - lr: 0.0010
Epoch 17/20
506/506 [==============================] - 9s 18ms/step - loss: 0.2704 - accuracy: 0.9162 - val_loss: 0.3404 - val_accuracy: 0.8791 - lr: 0.0010
Epoch 18/20
506/506 [==============================] - 9s 18ms/step - loss: 0.2612 - accuracy: 0.9187 - val_loss: 0.3486 - val_accuracy: 0.8816 - lr: 0.0010
Epoch 19/20
506/506 [==============================] - 9s 18ms/step -

2023-09-19 10:16:57.871043: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./model/bestmodel: FAILED_PRECONDITION: model/bestmodel; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


125/125 [==============================] - 1s 6ms/step
PLeccNET
TP: 1785 FP: 214 TN: 1722 FN: 277
ACC: 0.8771885942971486 F1: 0.8790938192563408 MCC: 0.7547521078358185


In [9]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPool1D, Flatten, Dropout, Conv1D,Reshape,Input,AveragePooling1D,LSTM,Attention,MultiHeadAttention,MaxPooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras import regularizers,Model,optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.losses import binary_crossentropy, hinge
callback4 = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,
                                             verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
                 callbacks.ModelCheckpoint(filepath=r'./model/bestmodel', monitor='val_accuracy', verbose=0,mode = 'max',
                                           save_best_only=True, save_weights_only=False)]
tf.random.set_seed(123)
np.random.seed(123)

lu = 200
f = open('./ecc/arabidopsis/'+str(lu)+'/arabidopsis.csv')
train_data = []
label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    dd = []
    for j in tt[0].upper():
        if(j == 'A'):
            dd.append([0,0,0,1])
        elif (j == 'G'):
            dd.append([0,0,1,0])
        elif (j == 'C'):
            dd.append([0,1,0,0])
        elif (j == 'T'):
            dd.append([1,0,0,0])
        else:
            dd.append([0,0,0,0])
    while(len(dd) < lu*2):
        dd.append([0,0,0,0])
    train_data.append(dd)

    label.append(int(tt[1]))



train_data = np.array(train_data)
label = np.array(label)





train_data,test_data,label,test_label=train_test_split(train_data,label,test_size=0.1,random_state=3753,stratify=label)

test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
label = np.array(label)
print(train_data.shape)
print(test_data.shape)




inputs1 = Input(shape=(lu*2,4),name='1st_input')








x_mod = Conv1D(128,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)

x_mod = Conv1D(64,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x_mod)

x_mod = MaxPooling1D(2)(x_mod)
x_mod = Dropout(0.1)(x_mod)
# x_mod = BatchNormalization()(x_mod)





co1 = Conv1D(x_mod.shape[1],4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
co2 = Conv1D(x_mod.shape[1],16, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co3 = Conv1D(247,6, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co4 = Conv1D(247,8, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co5 = Conv1D(247,10, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
pool1 = GlobalMaxPooling1D()(co1)
pool2 = GlobalMaxPooling1D()(co2)
# pool3 = GlobalMaxPooling1D()(co3)
# pool4 = GlobalMaxPooling1D()(co4)
# pool5 = GlobalMaxPooling1D()(co5)

# 使用FPN结构合并多尺度特征
fn = tf.add(pool1, pool2)
fn = Reshape((x_mod.shape[1], -1))(fn)


x_mod = Concatenate()([fn, x_mod])
x_mod = LSTM(64,return_sequences=True)(x_mod)


x_mod = Dropout(0.1)(x_mod)


x_mod = Flatten()(x_mod)
x_mod = Dropout(0.1)(x_mod)
x_mod = Dense(64, activation='sigmoid')(x_mod)
out = Dense(1, activation='sigmoid')(x_mod)

model = Model(inputs=[inputs1], outputs=[out])#合成模型


# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 交叉熵损失
    cross_entropy_loss = binary_crossentropy(y_true, y_pred)
    
    # Hinge Cross-Entropy Loss
    hinge_loss = tf.losses.hinge(y_true, y_pred)
    
    # 将两个损失相加
    total_loss = 0.95*cross_entropy_loss + 0.05*hinge_loss
    
    return total_loss




model.compile(loss=custom_loss,
              optimizer=optimizers.Adam(learning_rate=0.001,beta_1=0.9, beta_2=0.888, epsilon=1e-08),metrics=['accuracy'],
              )



train_history = model.fit(train_data,
                          label,
                          validation_split=0.1,
                          epochs=20,
                          batch_size=64,
                          verbose=1,
                          shuffle=True,
                          callbacks = callback4)


model.load_weights('./model/bestmodel')
pl = model.predict(test_data)
p_test_label = []

count = 0
for i in range(len(pl)):
    if(pl[i] >= 0.5):
        p_test_label.append(1)
    else:
        p_test_label.append(0)
# print(p_test_label)
TP, FP, TN, FN, ACC, F1, MCC = comparison(p_test_label, test_label)
print('PLeccNET')
print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN)
print('ACC:', ACC, 'F1:', F1, 'MCC:', MCC) 



(35980, 400, 4)
(3998, 400, 4)
Epoch 1/20
506/506 [==============================] - ETA: 0s - loss: 0.4826 - accuracy: 0.8019INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 21s 33ms/step - loss: 0.4826 - accuracy: 0.8019 - val_loss: 0.4123 - val_accuracy: 0.8658 - lr: 0.0010
Epoch 2/20
505/506 [============================>.] - ETA: 0s - loss: 0.3880 - accuracy: 0.8597INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 15s 31ms/step - loss: 0.3878 - accuracy: 0.8599 - val_loss: 0.3434 - val_accuracy: 0.8822 - lr: 0.0010
Epoch 3/20
506/506 [==============================] - 11s 22ms/step - loss: 0.3582 - accuracy: 0.8722 - val_loss: 0.3452 - val_accuracy: 0.8799 - lr: 0.0010
Epoch 4/20
506/506 [==============================] - 11s 22ms/step - loss: 0.3333 - accuracy: 0.8848 - val_loss: 0.3421 - val_accuracy: 0.8785 - lr: 0.0010
Epoch 5/20
505/506 [============================>.] - ETA: 0s - loss: 0.3140 - accuracy: 0.8934INFO:tensorflow:Assets written to: ./model/bestmodel/assets


...

506/506 [==============================] - 15s 31ms/step - loss: 0.2883 - accuracy: 0.9063 - val_loss: 0.2957 - val_accuracy: 0.9011 - lr: 0.0010
Epoch 11/20
506/506 [==============================] - 11s 22ms/step - loss: 0.2666 - accuracy: 0.9160 - val_loss: 0.3050 - val_accuracy: 0.8999 - lr: 0.0010
Epoch 12/20
506/506 [==============================] - 11s 22ms/step - loss: 0.2580 - accuracy: 0.9195 - val_loss: 0.3257 - val_accuracy: 0.8880 - lr: 0.0010
Epoch 13/20
505/506 [============================>.] - ETA: 0s - loss: 0.2601 - accuracy: 0.9184INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 15s 31ms/step - loss: 0.2600 - accuracy: 0.9185 - val_loss: 0.2911 - val_accuracy: 0.9049 - lr: 0.0010
Epoch 14/20
506/506 [==============================] - 11s 22ms/step - loss: 0.2520 - accuracy: 0.9225 - val_loss: 0.3085 - val_accuracy: 0.8963 - lr: 0.0010
Epoch 15/20
506/506 [==============================] - 11s 22ms/step - loss: 0.2477 - accuracy: 0.9254 - val_loss: 0.3404 - val_accuracy: 0.8877 - lr: 0.0010
Epoch 16/20
506/506 [==============================] - 11s 22ms/step - loss: 0.2415 - accuracy: 0.9271 - val_loss: 0.3864 - val_accuracy: 0.8616 - lr: 0.0010
Epoch 17/20
506/506 [==============================] - 11s 22ms/step - loss: 0.2381 - accuracy: 0.9292 - val_loss: 0.4303 - val_accuracy: 0.8524 - lr: 0.0010
Epoch 18/20
506/506 [==============================] - 11s 22ms/step - loss: 0.2305 - accuracy: 0.9324 - val_loss: 0.2942 - val_accuracy: 0.9036 - lr: 0.0010
Epoch 19/20
506/506 [==============================] - 11s 22ms/

2023-09-19 10:22:14.120209: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./model/bestmodel: FAILED_PRECONDITION: model/bestmodel; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


125/125 [==============================] - 1s 7ms/step
PLeccNET
TP: 1702 FP: 297 TN: 1878 FN: 121
ACC: 0.8954477238619309 F1: 0.8906331763474621 MCC: 0.7939787971830401


In [10]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPool1D, Flatten, Dropout, Conv1D,Reshape,Input,AveragePooling1D,LSTM,Attention,MultiHeadAttention,MaxPooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras import regularizers,Model,optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.losses import binary_crossentropy, hinge
callback4 = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,
                                             verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
                 callbacks.ModelCheckpoint(filepath=r'./model/bestmodel', monitor='val_accuracy', verbose=0,mode = 'max',
                                           save_best_only=True, save_weights_only=False)]
tf.random.set_seed(123)
np.random.seed(123)

lu = 300
f = open('./ecc/arabidopsis/'+str(lu)+'/arabidopsis.csv')
train_data = []
label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    dd = []
    for j in tt[0].upper():
        if(j == 'A'):
            dd.append([0,0,0,1])
        elif (j == 'G'):
            dd.append([0,0,1,0])
        elif (j == 'C'):
            dd.append([0,1,0,0])
        elif (j == 'T'):
            dd.append([1,0,0,0])
        else:
            dd.append([0,0,0,0])
    while(len(dd) < lu*2):
        dd.append([0,0,0,0])
    train_data.append(dd)

    label.append(int(tt[1]))



train_data = np.array(train_data)
label = np.array(label)





train_data,test_data,label,test_label=train_test_split(train_data,label,test_size=0.1,random_state=3753,stratify=label)

test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
label = np.array(label)
print(train_data.shape)
print(test_data.shape)




inputs1 = Input(shape=(lu*2,4),name='1st_input')








x_mod = Conv1D(128,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)

x_mod = Conv1D(64,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x_mod)

x_mod = MaxPooling1D(2)(x_mod)
x_mod = Dropout(0.1)(x_mod)
# x_mod = BatchNormalization()(x_mod)





co1 = Conv1D(x_mod.shape[1],4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
co2 = Conv1D(x_mod.shape[1],16, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co3 = Conv1D(247,6, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co4 = Conv1D(247,8, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co5 = Conv1D(247,10, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
pool1 = GlobalMaxPooling1D()(co1)
pool2 = GlobalMaxPooling1D()(co2)
# pool3 = GlobalMaxPooling1D()(co3)
# pool4 = GlobalMaxPooling1D()(co4)
# pool5 = GlobalMaxPooling1D()(co5)

# 使用FPN结构合并多尺度特征
fn = tf.add(pool1, pool2)
fn = Reshape((x_mod.shape[1], -1))(fn)


x_mod = Concatenate()([fn, x_mod])
x_mod = LSTM(64,return_sequences=True)(x_mod)


x_mod = Dropout(0.1)(x_mod)


x_mod = Flatten()(x_mod)
x_mod = Dropout(0.1)(x_mod)
x_mod = Dense(64, activation='sigmoid')(x_mod)
out = Dense(1, activation='sigmoid')(x_mod)

model = Model(inputs=[inputs1], outputs=[out])#合成模型


# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 交叉熵损失
    cross_entropy_loss = binary_crossentropy(y_true, y_pred)
    
    # Hinge Cross-Entropy Loss
    hinge_loss = tf.losses.hinge(y_true, y_pred)
    
    # 将两个损失相加
    total_loss = 0.95*cross_entropy_loss + 0.05*hinge_loss
    
    return total_loss




model.compile(loss=custom_loss,
              optimizer=optimizers.Adam(learning_rate=0.001,beta_1=0.9, beta_2=0.888, epsilon=1e-08),metrics=['accuracy'],
              )



train_history = model.fit(train_data,
                          label,
                          validation_split=0.1,
                          epochs=20,
                          batch_size=64,
                          verbose=1,
                          shuffle=True,
                          callbacks = callback4)


model.load_weights('./model/bestmodel')
pl = model.predict(test_data)
p_test_label = []

count = 0
for i in range(len(pl)):
    if(pl[i] >= 0.5):
        p_test_label.append(1)
    else:
        p_test_label.append(0)
# print(p_test_label)
TP, FP, TN, FN, ACC, F1, MCC = comparison(p_test_label, test_label)
print('PLeccNET')
print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN)
print('ACC:', ACC, 'F1:', F1, 'MCC:', MCC) 



(35980, 600, 4)
(3998, 600, 4)
Epoch 1/20
506/506 [==============================] - ETA: 0s - loss: 0.4469 - accuracy: 0.8090INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 27s 44ms/step - loss: 0.4469 - accuracy: 0.8090 - val_loss: 0.3179 - val_accuracy: 0.8936 - lr: 0.0010
Epoch 2/20
505/506 [============================>.] - ETA: 0s - loss: 0.2892 - accuracy: 0.9004INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 22s 43ms/step - loss: 0.2890 - accuracy: 0.9005 - val_loss: 0.2741 - val_accuracy: 0.8986 - lr: 0.0010
Epoch 3/20
505/506 [============================>.] - ETA: 0s - loss: 0.2441 - accuracy: 0.9189INFO:tensorflow:Assets written to: ./model/bestmodel/assets


...

506/506 [==============================] - 22s 43ms/step - loss: 0.2344 - accuracy: 0.9232 - val_loss: 0.2647 - val_accuracy: 0.9111 - lr: 0.0010
Epoch 5/20
506/506 [==============================] - 17s 33ms/step - loss: 0.2143 - accuracy: 0.9337 - val_loss: 0.4166 - val_accuracy: 0.8588 - lr: 0.0010
Epoch 6/20
506/506 [==============================] - 17s 33ms/step - loss: 0.2087 - accuracy: 0.9362 - val_loss: 0.3366 - val_accuracy: 0.8813 - lr: 0.0010
Epoch 7/20
505/506 [============================>.] - ETA: 0s - loss: 0.2006 - accuracy: 0.9430INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 21s 42ms/step - loss: 0.2009 - accuracy: 0.9429 - val_loss: 0.2717 - val_accuracy: 0.9188 - lr: 0.0010
Epoch 8/20
506/506 [==============================] - 17s 33ms/step - loss: 0.1933 - accuracy: 0.9463 - val_loss: 0.2843 - val_accuracy: 0.9138 - lr: 0.0010
Epoch 9/20
506/506 [==============================] - 17s 33ms/step - loss: 0.1914 - accuracy: 0.9480 - val_loss: 0.3015 - val_accuracy: 0.9038 - lr: 0.0010
Epoch 10/20
506/506 [==============================] - 17s 33ms/step - loss: 0.1948 - accuracy: 0.9463 - val_loss: 0.3145 - val_accuracy: 0.8922 - lr: 0.0010
Epoch 11/20
506/506 [==============================] - 17s 33ms/step - loss: 0.2036 - accuracy: 0.9430 - val_loss: 0.3810 - val_accuracy: 0.8941 - lr: 0.0010
Epoch 12/20
506/506 [==============================] - 17s 33ms/step - loss: 0.1844 - accuracy: 0.9518 - val_loss: 0.3320 - val_accuracy: 0.8961 - lr: 0.0010
Epoch 13/20
506/506 [==============================] - 17s 33ms/st

2023-09-19 10:30:16.822609: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./model/bestmodel: FAILED_PRECONDITION: model/bestmodel; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


125/125 [==============================] - 2s 8ms/step
PLeccNET
TP: 1825 FP: 174 TN: 1825 FN: 174
ACC: 0.9129564782391195 F1: 0.9129564782391195 MCC: 0.8259129564782391


In [11]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPool1D, Flatten, Dropout, Conv1D,Reshape,Input,AveragePooling1D,LSTM,Attention,MultiHeadAttention,MaxPooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras import regularizers,Model,optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.losses import binary_crossentropy, hinge
callback4 = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,
                                             verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
                 callbacks.ModelCheckpoint(filepath=r'./model/bestmodel', monitor='val_accuracy', verbose=0,mode = 'max',
                                           save_best_only=True, save_weights_only=False)]
tf.random.set_seed(123)
np.random.seed(123)

lu = 350
f = open('./ecc/arabidopsis/'+str(lu)+'/arabidopsis.csv')
train_data = []
label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    dd = []
    for j in tt[0].upper():
        if(j == 'A'):
            dd.append([0,0,0,1])
        elif (j == 'G'):
            dd.append([0,0,1,0])
        elif (j == 'C'):
            dd.append([0,1,0,0])
        elif (j == 'T'):
            dd.append([1,0,0,0])
        else:
            dd.append([0,0,0,0])
    while(len(dd) < lu*2):
        dd.append([0,0,0,0])
    train_data.append(dd)

    label.append(int(tt[1]))



train_data = np.array(train_data)
label = np.array(label)





train_data,test_data,label,test_label=train_test_split(train_data,label,test_size=0.1,random_state=3753,stratify=label)

test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
label = np.array(label)
print(train_data.shape)
print(test_data.shape)




inputs1 = Input(shape=(lu*2,4),name='1st_input')








x_mod = Conv1D(128,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)

x_mod = Conv1D(64,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x_mod)

x_mod = MaxPooling1D(2)(x_mod)
x_mod = Dropout(0.1)(x_mod)
# x_mod = BatchNormalization()(x_mod)





co1 = Conv1D(x_mod.shape[1],4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
co2 = Conv1D(x_mod.shape[1],16, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co3 = Conv1D(247,6, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co4 = Conv1D(247,8, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co5 = Conv1D(247,10, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
pool1 = GlobalMaxPooling1D()(co1)
pool2 = GlobalMaxPooling1D()(co2)
# pool3 = GlobalMaxPooling1D()(co3)
# pool4 = GlobalMaxPooling1D()(co4)
# pool5 = GlobalMaxPooling1D()(co5)

# 使用FPN结构合并多尺度特征
fn = tf.add(pool1, pool2)
fn = Reshape((x_mod.shape[1], -1))(fn)


x_mod = Concatenate()([fn, x_mod])
x_mod = LSTM(64,return_sequences=True)(x_mod)


x_mod = Dropout(0.1)(x_mod)


x_mod = Flatten()(x_mod)
x_mod = Dropout(0.1)(x_mod)
x_mod = Dense(64, activation='sigmoid')(x_mod)
out = Dense(1, activation='sigmoid')(x_mod)

model = Model(inputs=[inputs1], outputs=[out])#合成模型


# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 交叉熵损失
    cross_entropy_loss = binary_crossentropy(y_true, y_pred)
    
    # Hinge Cross-Entropy Loss
    hinge_loss = tf.losses.hinge(y_true, y_pred)
    
    # 将两个损失相加
    total_loss = 0.95*cross_entropy_loss + 0.05*hinge_loss
    
    return total_loss




model.compile(loss=custom_loss,
              optimizer=optimizers.Adam(learning_rate=0.001,beta_1=0.9, beta_2=0.888, epsilon=1e-08),metrics=['accuracy'],
              )



train_history = model.fit(train_data,
                          label,
                          validation_split=0.1,
                          epochs=20,
                          batch_size=64,
                          verbose=1,
                          shuffle=True,
                          callbacks = callback4)


model.load_weights('./model/bestmodel')
pl = model.predict(test_data)
p_test_label = []

count = 0
for i in range(len(pl)):
    if(pl[i] >= 0.5):
        p_test_label.append(1)
    else:
        p_test_label.append(0)
# print(p_test_label)
TP, FP, TN, FN, ACC, F1, MCC = comparison(p_test_label, test_label)
print('PLeccNET')
print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN)
print('ACC:', ACC, 'F1:', F1, 'MCC:', MCC) 



(35980, 700, 4)
(3998, 700, 4)
Epoch 1/20
506/506 [==============================] - ETA: 0s - loss: 0.4533 - accuracy: 0.8105INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 30s 51ms/step - loss: 0.4533 - accuracy: 0.8105 - val_loss: 0.3720 - val_accuracy: 0.8602 - lr: 0.0010
Epoch 2/20
505/506 [============================>.] - ETA: 0s - loss: 0.3349 - accuracy: 0.8844INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 25s 49ms/step - loss: 0.3348 - accuracy: 0.8844 - val_loss: 0.3292 - val_accuracy: 0.8877 - lr: 0.0010
Epoch 3/20
506/506 [==============================] - 20s 39ms/step - loss: 0.6729 - accuracy: 0.6474 - val_loss: 0.7089 - val_accuracy: 0.4956 - lr: 0.0010
Epoch 4/20
506/506 [==============================] - 20s 40ms/step - loss: 0.7090 - accuracy: 0.5019 - val_loss: 0.7082 - val_accuracy: 0.5044 - lr: 0.0010
Epoch 5/20
506/506 [==============================] - 20s 39ms/step - loss: 0.7090 - accuracy: 0.4969 - val_loss: 0.7083 - val_accuracy: 0.4956 - lr: 0.0010
Epoch 6/20
506/506 [==============================] - 20s 39ms/step - loss: 0.7092 - accuracy: 0.4993 - val_loss: 0.7088 - val_accuracy: 0.5044 - lr: 0.0010
Epoch 7/20
506/506 [==============================] - 20s 39ms/step - loss: 0.7092 - accuracy: 0.5001 - val_loss: 0.7089 - val_accuracy: 0.4956 - lr: 0.0010
Epoch 8/20
506/506 [==============================] - 20s 39ms/step -

2023-09-19 10:39:08.564874: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./model/bestmodel: FAILED_PRECONDITION: model/bestmodel; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


125/125 [==============================] - 2s 9ms/step
PLeccNET
TP: 1602 FP: 397 TN: 1935 FN: 64
ACC: 0.8846923461730866 F1: 0.8742155525238745 MCC: 0.7802873458420673


In [12]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPool1D, Flatten, Dropout, Conv1D,Reshape,Input,AveragePooling1D,LSTM,Attention,MultiHeadAttention,MaxPooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras import regularizers,Model,optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.losses import binary_crossentropy, hinge
callback4 = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,
                                             verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
                 callbacks.ModelCheckpoint(filepath=r'./model/bestmodel', monitor='val_accuracy', verbose=0,mode = 'max',
                                           save_best_only=True, save_weights_only=False)]
tf.random.set_seed(123)
np.random.seed(123)

lu = 400
f = open('./ecc/arabidopsis/'+str(lu)+'/arabidopsis.csv')
train_data = []
label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    dd = []
    for j in tt[0].upper():
        if(j == 'A'):
            dd.append([0,0,0,1])
        elif (j == 'G'):
            dd.append([0,0,1,0])
        elif (j == 'C'):
            dd.append([0,1,0,0])
        elif (j == 'T'):
            dd.append([1,0,0,0])
        else:
            dd.append([0,0,0,0])
    while(len(dd) < lu*2):
        dd.append([0,0,0,0])
    train_data.append(dd)

    label.append(int(tt[1]))



train_data = np.array(train_data)
label = np.array(label)





train_data,test_data,label,test_label=train_test_split(train_data,label,test_size=0.1,random_state=3753,stratify=label)

test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
label = np.array(label)
print(train_data.shape)
print(test_data.shape)




inputs1 = Input(shape=(lu*2,4),name='1st_input')








x_mod = Conv1D(128,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)

x_mod = Conv1D(64,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x_mod)

x_mod = MaxPooling1D(2)(x_mod)
x_mod = Dropout(0.1)(x_mod)
# x_mod = BatchNormalization()(x_mod)





co1 = Conv1D(x_mod.shape[1],4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
co2 = Conv1D(x_mod.shape[1],16, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co3 = Conv1D(247,6, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co4 = Conv1D(247,8, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co5 = Conv1D(247,10, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
pool1 = GlobalMaxPooling1D()(co1)
pool2 = GlobalMaxPooling1D()(co2)
# pool3 = GlobalMaxPooling1D()(co3)
# pool4 = GlobalMaxPooling1D()(co4)
# pool5 = GlobalMaxPooling1D()(co5)

# 使用FPN结构合并多尺度特征
fn = tf.add(pool1, pool2)
fn = Reshape((x_mod.shape[1], -1))(fn)


x_mod = Concatenate()([fn, x_mod])
x_mod = LSTM(64,return_sequences=True)(x_mod)


x_mod = Dropout(0.1)(x_mod)


x_mod = Flatten()(x_mod)
x_mod = Dropout(0.1)(x_mod)
x_mod = Dense(64, activation='sigmoid')(x_mod)
out = Dense(1, activation='sigmoid')(x_mod)

model = Model(inputs=[inputs1], outputs=[out])#合成模型


# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 交叉熵损失
    cross_entropy_loss = binary_crossentropy(y_true, y_pred)
    
    # Hinge Cross-Entropy Loss
    hinge_loss = tf.losses.hinge(y_true, y_pred)
    
    # 将两个损失相加
    total_loss = 0.95*cross_entropy_loss + 0.05*hinge_loss
    
    return total_loss




model.compile(loss=custom_loss,
              optimizer=optimizers.Adam(learning_rate=0.001,beta_1=0.9, beta_2=0.888, epsilon=1e-08),metrics=['accuracy'],
              )



train_history = model.fit(train_data,
                          label,
                          validation_split=0.1,
                          epochs=20,
                          batch_size=64,
                          verbose=1,
                          shuffle=True,
                          callbacks = callback4)


model.load_weights('./model/bestmodel')
pl = model.predict(test_data)
p_test_label = []

count = 0
for i in range(len(pl)):
    if(pl[i] >= 0.5):
        p_test_label.append(1)
    else:
        p_test_label.append(0)
# print(p_test_label)
TP, FP, TN, FN, ACC, F1, MCC = comparison(p_test_label, test_label)
print('PLeccNET')
print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN)
print('ACC:', ACC, 'F1:', F1, 'MCC:', MCC) 



(35980, 800, 4)
(3998, 800, 4)
Epoch 1/20
506/506 [==============================] - ETA: 0s - loss: 0.4391 - accuracy: 0.8154INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 34s 57ms/step - loss: 0.4391 - accuracy: 0.8154 - val_loss: 0.2976 - val_accuracy: 0.9080 - lr: 0.0010
Epoch 2/20
506/506 [==============================] - 23s 46ms/step - loss: 0.7927 - accuracy: 0.6190 - val_loss: 0.7082 - val_accuracy: 0.5044 - lr: 0.0010
Epoch 3/20
506/506 [==============================] - 24s 47ms/step - loss: 0.7090 - accuracy: 0.5016 - val_loss: 0.7087 - val_accuracy: 0.4956 - lr: 0.0010
Epoch 4/20
506/506 [==============================] - 23s 46ms/step - loss: 0.7090 - accuracy: 0.5021 - val_loss: 0.7082 - val_accuracy: 0.5044 - lr: 0.0010
Epoch 5/20
506/506 [==============================] - 23s 46ms/step - loss: 0.7092 - accuracy: 0.5001 - val_loss: 0.7089 - val_accuracy: 0.4956 - lr: 0.0010
Epoch 8/20
506/506 [==============================] - 23s 46ms/step - loss: 0.7092 - accuracy: 0.5001 - val_loss: 0.7083 - val_accuracy: 0.5044 - lr: 0.0010
Epoch 9/20
506/506 [==============================] - 23s 46ms/step -

2023-09-19 10:58:08.201382: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./model/bestmodel: FAILED_PRECONDITION: model/bestmodel; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


125/125 [==============================] - 2s 11ms/step
PLeccNET
TP: 1824 FP: 175 TN: 1777 FN: 222
ACC: 0.9007003501750875 F1: 0.9018541409147095 MCC: 0.8016223004514819


In [14]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPool1D, Flatten, Dropout, Conv1D,Reshape,Input,AveragePooling1D,LSTM,Attention,MultiHeadAttention,MaxPooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras import regularizers,Model,optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.losses import binary_crossentropy, hinge
callback4 = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,
                                             verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
                 callbacks.ModelCheckpoint(filepath=r'./model/bestmodel', monitor='val_accuracy', verbose=0,mode = 'max',
                                           save_best_only=True, save_weights_only=False)]
tf.random.set_seed(123)
np.random.seed(123)

lu = 450
f = open('./ecc/arabidopsis/'+str(lu)+'/arabidopsis.csv')
train_data = []
label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    dd = []
    for j in tt[0].upper():
        if(j == 'A'):
            dd.append([0,0,0,1])
        elif (j == 'G'):
            dd.append([0,0,1,0])
        elif (j == 'C'):
            dd.append([0,1,0,0])
        elif (j == 'T'):
            dd.append([1,0,0,0])
        else:
            dd.append([0,0,0,0])
    while(len(dd) < lu*2):
        dd.append([0,0,0,0])
    train_data.append(dd)

    label.append(int(tt[1]))



train_data = np.array(train_data)
label = np.array(label)





train_data,test_data,label,test_label=train_test_split(train_data,label,test_size=0.1,random_state=3753,stratify=label)

test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
label = np.array(label)
print(train_data.shape)
print(test_data.shape)




inputs1 = Input(shape=(lu*2,4),name='1st_input')








x_mod = Conv1D(128,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)

x_mod = Conv1D(64,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x_mod)

x_mod = MaxPooling1D(2)(x_mod)
x_mod = Dropout(0.1)(x_mod)
# x_mod = BatchNormalization()(x_mod)





co1 = Conv1D(x_mod.shape[1],4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
co2 = Conv1D(x_mod.shape[1],16, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co3 = Conv1D(247,6, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co4 = Conv1D(247,8, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co5 = Conv1D(247,10, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
pool1 = GlobalMaxPooling1D()(co1)
pool2 = GlobalMaxPooling1D()(co2)
# pool3 = GlobalMaxPooling1D()(co3)
# pool4 = GlobalMaxPooling1D()(co4)
# pool5 = GlobalMaxPooling1D()(co5)

# 使用FPN结构合并多尺度特征
fn = tf.add(pool1, pool2)
fn = Reshape((x_mod.shape[1], -1))(fn)


x_mod = Concatenate()([fn, x_mod])
x_mod = LSTM(64,return_sequences=True)(x_mod)


x_mod = Dropout(0.1)(x_mod)


x_mod = Flatten()(x_mod)
x_mod = Dropout(0.1)(x_mod)
x_mod = Dense(64, activation='sigmoid')(x_mod)
out = Dense(1, activation='sigmoid')(x_mod)

model = Model(inputs=[inputs1], outputs=[out])#合成模型


# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 交叉熵损失
    cross_entropy_loss = binary_crossentropy(y_true, y_pred)
    
    # Hinge Cross-Entropy Loss
    hinge_loss = tf.losses.hinge(y_true, y_pred)
    
    # 将两个损失相加
    total_loss = 0.95*cross_entropy_loss + 0.05*hinge_loss
    
    return total_loss




model.compile(loss=custom_loss,
              optimizer=optimizers.Adam(learning_rate=0.001,beta_1=0.9, beta_2=0.888, epsilon=1e-08),metrics=['accuracy'],
              )



train_history = model.fit(train_data,
                          label,
                          validation_split=0.1,
                          epochs=20,
                          batch_size=64,
                          verbose=1,
                          shuffle=True,
                          callbacks = callback4)


model.load_weights('./model/bestmodel')
pl = model.predict(test_data)
p_test_label = []

count = 0
for i in range(len(pl)):
    if(pl[i] >= 0.5):
        p_test_label.append(1)
    else:
        p_test_label.append(0)
# print(p_test_label)
TP, FP, TN, FN, ACC, F1, MCC = comparison(p_test_label, test_label)
print('PLeccNET')
print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN)
print('ACC:', ACC, 'F1:', F1, 'MCC:', MCC) 



(35980, 900, 4)
(3998, 900, 4)
Epoch 1/20
506/506 [==============================] - ETA: 0s - loss: 0.4415 - accuracy: 0.8142INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 36s 64ms/step - loss: 0.4415 - accuracy: 0.8142 - val_loss: 0.3148 - val_accuracy: 0.8908 - lr: 0.0010
Epoch 2/20
506/506 [==============================] - ETA: 0s - loss: 0.2990 - accuracy: 0.8978INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 32s 63ms/step - loss: 0.2990 - accuracy: 0.8978 - val_loss: 0.2503 - val_accuracy: 0.9197 - lr: 0.0010
Epoch 3/20
506/506 [==============================] - 27s 53ms/step - loss: 0.2683 - accuracy: 0.9099 - val_loss: 0.3280 - val_accuracy: 0.8766 - lr: 0.0010
Epoch 4/20
506/506 [==============================] - ETA: 0s - loss: 0.2427 - accuracy: 0.9220INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 32s 62ms/step - loss: 0.2427 - accuracy: 0.9220 - val_loss: 0.2394 - val_accuracy: 0.9261 - lr: 0.0010
Epoch 5/20
506/506 [==============================] - 27s 53ms/step - loss: 0.3291 - accuracy: 0.8450 - val_loss: 0.7224 - val_accuracy: 0.4956 - lr: 0.0010
Epoch 6/20
506/506 [==============================] - 27s 53ms/step - loss: 0.6745 - accuracy: 0.5890 - val_loss: 0.5196 - val_accuracy: 0.7640 - lr: 0.0010
Epoch 7/20
506/506 [==============================] - 27s 53ms/step - loss: 0.5790 - accuracy: 0.7052 - val_loss: 0.3189 - val_accuracy: 0.8769 - lr: 0.0010
Epoch 8/20
506/506 [==============================] - 27s 53ms/step - loss: 0.2997 - accuracy: 0.8863 - val_loss: 0.2693 - val_accuracy: 0.9033 - lr: 0.0010
Epoch 9/20
506/506 [==============================] - 27s 54ms/step - loss: 0.2685 - accuracy: 0.9023 - val_loss: 0.2600 - val_accuracy: 0.9024 - lr: 0.0010
Epoch 10/20
506/506 [==============================] - 27s 53ms/step 

2023-09-19 11:24:13.988303: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./model/bestmodel: FAILED_PRECONDITION: model/bestmodel; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


125/125 [==============================] - 2s 12ms/step
PLeccNET
TP: 1857 FP: 142 TN: 1795 FN: 204
ACC: 0.9134567283641821 F1: 0.9147783251231527 MCC: 0.8273114734551081


In [15]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPool1D, Flatten, Dropout, Conv1D,Reshape,Input,AveragePooling1D,LSTM,Attention,MultiHeadAttention,MaxPooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras import regularizers,Model,optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from tensorflow.keras.losses import binary_crossentropy, hinge
callback4 = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,
                                             verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
                 callbacks.ModelCheckpoint(filepath=r'./model/bestmodel', monitor='val_accuracy', verbose=0,mode = 'max',
                                           save_best_only=True, save_weights_only=False)]
tf.random.set_seed(123)
np.random.seed(123)

lu = 500
f = open('./ecc/arabidopsis/'+str(lu)+'/arabidopsis.csv')
train_data = []
label = []
while(True):
    tt = f.readline()[:-1]
    if(len(tt) <= 1):
        break
    tt = tt.split(',')
    dd = []
    for j in tt[0].upper():
        if(j == 'A'):
            dd.append([0,0,0,1])
        elif (j == 'G'):
            dd.append([0,0,1,0])
        elif (j == 'C'):
            dd.append([0,1,0,0])
        elif (j == 'T'):
            dd.append([1,0,0,0])
        else:
            dd.append([0,0,0,0])
    while(len(dd) < lu*2):
        dd.append([0,0,0,0])
    train_data.append(dd)

    label.append(int(tt[1]))



train_data = np.array(train_data)
label = np.array(label)





train_data,test_data,label,test_label=train_test_split(train_data,label,test_size=0.1,random_state=3753,stratify=label)

test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
label = np.array(label)
print(train_data.shape)
print(test_data.shape)




inputs1 = Input(shape=(lu*2,4),name='1st_input')








x_mod = Conv1D(128,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)

x_mod = Conv1D(64,4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x_mod)

x_mod = MaxPooling1D(2)(x_mod)
x_mod = Dropout(0.1)(x_mod)
# x_mod = BatchNormalization()(x_mod)





co1 = Conv1D(x_mod.shape[1],4, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
co2 = Conv1D(x_mod.shape[1],16, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co3 = Conv1D(247,6, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co4 = Conv1D(247,8, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
# co5 = Conv1D(247,10, activation='relu',kernel_regularizer=regularizers.l2(0.001))(inputs1)
pool1 = GlobalMaxPooling1D()(co1)
pool2 = GlobalMaxPooling1D()(co2)
# pool3 = GlobalMaxPooling1D()(co3)
# pool4 = GlobalMaxPooling1D()(co4)
# pool5 = GlobalMaxPooling1D()(co5)

# 使用FPN结构合并多尺度特征
fn = tf.add(pool1, pool2)
fn = Reshape((x_mod.shape[1], -1))(fn)


x_mod = Concatenate()([fn, x_mod])
x_mod = LSTM(64,return_sequences=True)(x_mod)


x_mod = Dropout(0.1)(x_mod)


x_mod = Flatten()(x_mod)
x_mod = Dropout(0.1)(x_mod)
x_mod = Dense(64, activation='sigmoid')(x_mod)
out = Dense(1, activation='sigmoid')(x_mod)

model = Model(inputs=[inputs1], outputs=[out])#合成模型


# 自定义损失函数
def custom_loss(y_true, y_pred):
    # 交叉熵损失
    cross_entropy_loss = binary_crossentropy(y_true, y_pred)
    
    # Hinge Cross-Entropy Loss
    hinge_loss = tf.losses.hinge(y_true, y_pred)
    
    # 将两个损失相加
    total_loss = 0.95*cross_entropy_loss + 0.05*hinge_loss
    
    return total_loss




model.compile(loss=custom_loss,
              optimizer=optimizers.Adam(learning_rate=0.001,beta_1=0.9, beta_2=0.888, epsilon=1e-08),metrics=['accuracy'],
              )



train_history = model.fit(train_data,
                          label,
                          validation_split=0.1,
                          epochs=20,
                          batch_size=64,
                          verbose=1,
                          shuffle=True,
                          callbacks = callback4)


model.load_weights('./model/bestmodel')
pl = model.predict(test_data)
p_test_label = []

count = 0
for i in range(len(pl)):
    if(pl[i] >= 0.5):
        p_test_label.append(1)
    else:
        p_test_label.append(0)
# print(p_test_label)
TP, FP, TN, FN, ACC, F1, MCC = comparison(p_test_label, test_label)
print('PLeccNET')
print('TP:', TP, 'FP:', FP, 'TN:', TN, 'FN:', FN)
print('ACC:', ACC, 'F1:', F1, 'MCC:', MCC) 



(35980, 1000, 4)
(3998, 1000, 4)
Epoch 1/20
506/506 [==============================] - ETA: 0s - loss: 0.4386 - accuracy: 0.8178INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 41s 71ms/step - loss: 0.4386 - accuracy: 0.8178 - val_loss: 0.3336 - val_accuracy: 0.8855 - lr: 0.0010
Epoch 2/20
506/506 [==============================] - 30s 60ms/step - loss: 0.3188 - accuracy: 0.8921 - val_loss: 0.7618 - val_accuracy: 0.4958 - lr: 0.0010
Epoch 3/20
506/506 [==============================] - 30s 60ms/step - loss: 0.6254 - accuracy: 0.6098 - val_loss: 0.3440 - val_accuracy: 0.8613 - lr: 0.0010
Epoch 4/20
506/506 [==============================] - 30s 60ms/step - loss: 0.3246 - accuracy: 0.8708 - val_loss: 0.3002 - val_accuracy: 0.8855 - lr: 0.0010
Epoch 5/20
506/506 [==============================] - ETA: 0s - loss: 0.2741 - accuracy: 0.8987INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 35s 69ms/step - loss: 0.2741 - accuracy: 0.8987 - val_loss: 0.2955 - val_accuracy: 0.8866 - lr: 0.0010
Epoch 6/20
506/506 [==============================] - 30s 60ms/step - loss: 0.2515 - accuracy: 0.9086 - val_loss: 0.3183 - val_accuracy: 0.8702 - lr: 0.0010
Epoch 7/20
506/506 [==============================] - ETA: 0s - loss: 0.2453 - accuracy: 0.9123INFO:tensorflow:Assets written to: ./model/bestmodel/assets


...

506/506 [==============================] - 35s 69ms/step - loss: 0.2054 - accuracy: 0.9339 - val_loss: 0.2746 - val_accuracy: 0.9041 - lr: 0.0010
Epoch 13/20
506/506 [==============================] - 30s 60ms/step - loss: 0.2010 - accuracy: 0.9370 - val_loss: 0.2730 - val_accuracy: 0.9033 - lr: 0.0010
Epoch 14/20
506/506 [==============================] - ETA: 0s - loss: 0.2058 - accuracy: 0.9338INFO:tensorflow:Assets written to: ./model/bestmodel/assets


INFO:tensorflow:Assets written to: ./model/bestmodel/assets


506/506 [==============================] - 35s 69ms/step - loss: 0.2058 - accuracy: 0.9338 - val_loss: 0.2705 - val_accuracy: 0.9091 - lr: 0.0010
Epoch 15/20
506/506 [==============================] - 31s 60ms/step - loss: 0.1919 - accuracy: 0.9406 - val_loss: 0.4023 - val_accuracy: 0.8646 - lr: 0.0010
Epoch 16/20
506/506 [==============================] - 30s 60ms/step - loss: 0.1959 - accuracy: 0.9389 - val_loss: 0.3353 - val_accuracy: 0.8808 - lr: 0.0010
Epoch 17/20
506/506 [==============================] - 30s 60ms/step - loss: 0.1900 - accuracy: 0.9411 - val_loss: 0.2836 - val_accuracy: 0.9005 - lr: 0.0010
Epoch 18/20
506/506 [==============================] - 30s 60ms/step - loss: 0.1878 - accuracy: 0.9442 - val_loss: 0.3235 - val_accuracy: 0.8986 - lr: 0.0010
Epoch 19/20
506/506 [==============================] - 30s 60ms/step - loss: 0.1860 - accuracy: 0.9453 - val_loss: 0.3067 - val_accuracy: 0.9033 - lr: 0.0010
Epoch 20/20
506/506 [==============================] - 30s 60ms/

2023-09-19 11:37:52.048221: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./model/bestmodel: FAILED_PRECONDITION: model/bestmodel; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


125/125 [==============================] - 2s 14ms/step
PLeccNET
TP: 1825 FP: 174 TN: 1797 FN: 202
ACC: 0.9059529764882441 F1: 0.9066070541480378 MCC: 0.8119856111080338
